In [184]:
initial_chord = [60, 64, 67, 71] #defines by midi number the notes of the chord given by the user

#silulating a melody created by the algorithm, p = picth and r = rhythm
p = [60, 62, 64, 65, 67, 69, 71, 72]
r = [1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0]

N = len(p)
L = sum(r)

print(N, L)#teste

8 6.5


In [185]:
#Here start the equation to define the fitness function on stability

#Cn returns 1 when the note is a chord note, else 0
#Eq.(4)
def C(p):
    return 1 if p in initial_chord else 0

#CT calculates a fraction of CHORD TONES
#Eq.(3)
def CT(p, r):
    list_CT = [] 
    for i in range(N):
        num = C(p[i]) * r[i]
        den = L
        list_CT.append(num/den)
    return sum(list_CT)


print(CT(p, r))#teste

0.6153846153846154


In [186]:
#Eq.(6)
def U(x): #Não entendi exatamente o que seria o valor do x
    return 1 if x>=0 else 0
    
#SM calculates te amount of step motion(midi value less than 2) in p
#Asigns a higher score in the range of [0.0, 1.0] to the melody that contains more step motions. 
#Eq.(5)
def SM(p):
    list_SM = []
    for i in range(N-1):
        num = U(2 - abs(p[i] - p[i+1]))
        den = N-1
        list_SM.append(num/den)
    return sum(list_SM)

print(SM(p))#teste

0.9999999999999998


In [187]:
#DL evaluates a fraction if three consecutive notes in descending lines of the pitch sequence p
#Returns 1 only if the three successive notes are descending    
#Eq.(7)
def DL(p):
    list_DL = []
    for i in range(N-2):
        num = U(p[i] - p[i+1] - 1) * U(p[i+1] - p[i+2] - 1)
        den = N-2
        list_DL.append(num/den)
    return sum(list_DL)

print(DL(p))#teste

0.0


In [188]:
#fitness_1 calculates which is the higher stability value and assigns it with alpha, beta, gamma
#A higher fitness value presents a better melody in terms of stability
#Eq.(2)
def fitness_1(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0):
    list_fitness_1 = []
    list_fitness_1.append(CT(p,r))
    list_fitness_1.append(SM(p))
    list_fitness_1.append(DL(p))
    fitness_1_sorted = sorted(list_fitness_1, reverse = True)
    print (fitness_1_sorted) #teste para ver a lista ordenada de maior para menor
    
    more_stability = fitness_1_sorted[0]
    midle_stability = fitness_1_sorted[1]
    less_stability = fitness_1_sorted[2]
    
    return ((alpha*more_stability) + (beta*midle_stability) + (gamma*less_stability)) # - (delta*P(p, r) #ainda não coloquei o delta pois etá dando erro mais para frente
    
       
print (fitness_1(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0))#teste

[0.9999999999999998, 0.6153846153846154, 0.0]
4.23076923076923


In [189]:
#Here start the equation to define the fitness function on tension

#NT computes a fraction of non-chord tone(notes that is not in initial_chord) of the melody
#Eq.(9)
def NT(p, r):
    return 1.0 - CT(p, r)

print (NT(p, r))#teste

0.3846153846153846


In [190]:
#LM measures a ratio of skip motion(notes that are connected by a midi value higher than 2)
#Returns 1 when two consecutive notes are connected by a skip motion
#Thus, LM(p) returns a higher value when the pitch vector p is comprised of more skip motions
#Eq.(10)
def LM(p):
    list_LM = []
    for i in range(N-1):
        num = U(abs(p[i] - p[i+1]) - 2)
        den = N-1
        list_LM.append(num/den)
    return sum(list_LM)

print (LM(p))#teste

0.7142857142857142


In [191]:
#AL evaluates a fraction of three successive notes in ascending lines (in contrast with DL(p))
#Eq.(11)
def AL(p):
    list_AL = []
    for i in range(N-2):
        num = U(p[i+2] - p[i+1] - 1) * U(p[i+1] - p[i] - 1)
        den = N-2
        list_AL.append(num/den)
    return sum(list_AL)

print (AL(p))#teste

0.9999999999999999


In [192]:
#fitness_2 calculates which is the higher tension value and assigns it with alpha, beta, gamma
#A higher fitness value presents more instability on the melody
#Eq.(8)
def fitness_2(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0):
    list_fitness_2 = []
    list_fitness_2.append(NT(p,r))
    list_fitness_2.append(LM(p))
    list_fitness_2.append(AL(p))
    fitness_2_sorted = sorted(list_fitness_2, reverse = True)
    print (fitness_2_sorted)#teste para ver a lista ordenada de maior para menor
    
    more_tension = fitness_2_sorted[0]
    midle_tension = fitness_2_sorted[1]
    less_tension = fitness_2_sorted[2]
    
    return ((alpha*more_tension) + (beta*midle_tension) + (gamma*less_tension)) #depois adiconar "+ delta*P(p, r)"
    
       
print (fitness_2(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0))#teste

[0.9999999999999999, 0.7142857142857142, 0.3846153846153846]
4.813186813186812


In [193]:
#Defining the rules on the plausible pleasant non-chord tones

#Rule 1: Non-chord tone on the strong beat should move to a chord tone by a step motion

#Eq.(14)
def Nn(p): #Aqui eu fiquei confuso pois na Eq.(14) o artigo informa uma nova letra N, esta "nova letra N" eu defini como "Nn"
    return 1 if p not in initial_chord else 0
    
#UT measures the number of unresolved non-chord tones on the strong beats
#S is a set of indices of the strong beats #Mas o artigo não apresenta como calcular este índice de S
#Eq.(13)
def UT(p):
    list_UT = []
    for i in range(S):
        num = (Nn(p[i]) * (Nn(p[i+1]) + U(abs(p[i] - p[i+1])) - 2))
        list_UT.append(num)
    return sum(list_UT)
        
print (UT(p))#teste

NameError: name 'S' is not defined

In [194]:
#Rule 2: All non-chord tones should be connected with at least one adjacent chord tone by a step motion

#Eq.(15)
def NCC(p):
    list_NCC = []
    for i in range(N):
        num = (Nn(p[i])) * (Nn(p[i-1]) + U(abs(p[i-1] - p[i]) - 2))
        list_NCC.append(num)
    return sum(list_NCC)

#OR 
#não entendi porque o artigo colaca "or" nesta equação

def NCC2(p):
    list_NCC2 = []
    for i in range(N):
        num = (Nn(p[i+1]) + U(abs(p[i] - p[i+1]) - 2)) #aqui teria que colocar o " (Nn(p[i])) * " no início da equação?
        list_NCC2.append(num)
    return sum(list_NCC2)
        
    
print (NCC(p))#teste
print (NCC2(p))#teste

2


IndexError: list index out of range

In [195]:
#Rule 3: Skip motion on the non-chord tones should not be over the third interval

#NCL Captures the number of intervals which are greater than the third with an adjacent non-chord tone
#Eq.(16)
def NCL(p):
    list_NCL = []
    for i in range(N):
        num = Nn(p[i]) * (U(4 - abs(p[i-1] - p[i])) + U(4 - abs(p[i] - p[i+1])))
        list_NCL.append(num)
    return sum(list_NCL)

print (NCL(p))#teste

IndexError: list index out of range

In [196]:
#Here start the defining the penalty function

#Minimizing MNC(p) can resolve a misused non-chord tones in the melody
#Eq.(17)
def MNC(p):
    return UT(p) + NCC(p) + NCL(p)
    
print (MNC(p))#teste

NameError: name 'S' is not defined

In [197]:
#AN investigate the number o avoid notes over the strong beats
#If both Nn(p[i]) and C(p[i] - 1) return 1, p[i] becomes a avoid note
#Eq.(18)
def AN(p):
    list_AN = []
    for i in range(S):
        num = Nn(p[i]) * C(p[i] - 1)
        list_AN.append(num)
    return sum(list_AN)

print (AN(p))#teste

NameError: name 'S' is not defined

In [198]:
#ON counts the number of measures(i.e., bar) that contains more than 50% of non-chord tones
#M = number of measures #Mas o artigo não informa como é calculado isso
#Eq.(19)
def ON(p, r):
    list_ON = []
    list_eq_a = []
    for j in range(M[i]) #aqui sei que está errado, mas não saberia como colocar esta equação em Python
        num = r[j] * C(p[j])
        den = D #O artigo não informa o que é este D
        list_eq_a.append(num/den)
        eq_a = sum(list_eq_a)
    for i in range(M):
        num = U * (0.5 - eq_a)

SyntaxError: invalid syntax (<ipython-input-198-42768276f61f>, line 7)

In [199]:
#BL(p) computes the number of over-leap intervals
#Eq.(20)
def BL(p):
    list_BL = []
    for i in range(N-1):
        num = U(16 - abs(p[i] - p[i+1]))
        list_BL.append(num)
    return sum(list_BL)

print (BL(p))#teste

7


In [200]:
#Final equation to calculate the penalty

#Eq.(12)
def P(p, r):
    return ((MNC(p)) + (AN(p)) + (ON(p)) + (BL(p))) 
    
print(P(p, r))#teste

NameError: name 'S' is not defined